In [2]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from scipy import spatial
from scipy.sparse.linalg import eigs

git_path = '/Users/optas/Documents/Git_Repos/'
sys.path.insert(0, git_path)

from tf_lab.model.vgg import vgg_m_conv
from tf_lab.fundamentals import fc_with_soft_max_layer

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from tensorflow.models.image.cifar10 import cifar10
cifar10.FLAGS.data_dir = '/Users/optas/DATA/Images/CIFAR_10/'
cifar10.maybe_download_and_extract()
NUM_CLASSES = 10

In [4]:
images, labels = cifar10.distorted_inputs()
paddings = [[0,0], [50, 50,], [50, 50], [0,0]]
images_big = tf.pad(images, paddings, "CONSTANT")

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [12]:
def cifar_10_vgg_classifier(in_signal):
    keep_prob = 0.5
    stddev = 5e-2
    layer = vgg_m_conv(in_signal, keep_prob)
    predictions = fc_with_soft_max_layer(layer, NUM_CLASSES, stddev)      
    return predictions

In [15]:
with tf.Graph().as_default() as g:    
    # Get images and labels for CIFAR-10.
    images, labels = cifar10.distorted_inputs()
    pred = cifar_10_vgg_classifier(images)
    #     fc_cv, fc = test_vgg_m_conv(images, 1, g)
    init = tf.initialize_all_variables()        
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(init)
    # Start the queue runners.
    tf.train.start_queue_runners(sess=sess)    
    print sess.run([pred])
    

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
[array([[  9.89372240e-29,   9.99751389e-01,   4.25720461e-35, ...,
          2.48523487e-04,   7.65052235e-08,   6.46529433e-19],
       [  5.58076707e-22,   3.49713904e-23,   1.40103731e-35, ...,
          9.08389919e-10,   1.72844185e-26,   7.37164357e-11],
       [  9.71022601e-27,   2.51064481e-23,   0.00000000e+00, ...,
          1.55498248e-36,   6.02137003e-15,   7.03709175e-37],
       ..., 
       [  5.50711560e-12,   2.81982924e-19,   0.00000000e+00, ...,
          4.71312746e-12,   1.00000000e+00,   1.39827672e-09],
       [  9.99974370e-01,   1.99023309e-09,   4.99290294e-16, ...,
          2.56190851e-05,   1.22772589e-14,   1.48472119e-14],
       [  1.66505838e-24,   7.60911569e-29,   1.34719100e-33, ...,
          2.27212929e-14,   1.00000000e+00,   5.11814809e-24]], dtype=float32)]


[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
